In [0]:
from pyspark.sql.types import StructType,StructField,IntegerType,StringType,DateType,TimestampType

customer_schema = StructType([
StructField("customer_id",IntegerType()),
StructField("customer_name",StringType()),
StructField("date_of_birth",DateType()),
StructField("telephone",StringType()),
StructField("email",StringType()),
StructField("member_since",DateType()),
StructField("current_timestamp",TimestampType()) 
]
)

In [0]:
customer_df=(spark.readStream.
    format("json")
    .schema(customer_schema)
    .load("/Volumes/gizmobox/landing/operational_data/customer_stream/"))
display(customer_df)


In [0]:
from pyspark.sql.functions import col, current_timestamp
customer_df=(spark.readStream.
    format("json")
    .schema(customer_schema)
    .load("/Volumes/gizmobox/landing/operational_data/customer_stream/"))
customer_transformed_df = (
    customer_df.withColumn("file_name", col("_metadata.file_name"))
    .withColumn("ingestion_date", current_timestamp())
)
display(customer_transformed_df)

In [0]:
streaming_query = (
                    customer_transformed_df.writeStream
                    .format("delta")
                    .option("checkpointLocation","/Volumes/gizmobox/landing/operational_data/customer_stream/_checkpoint_stream"
    )
    .toTable("gizmobox.bronze.customer_stream")
)

In [0]:
%sql
select count(*) from gizmobox.bronze.customer_stream

In [0]:
streaming_query.stop()